# Stepper dancer

In [55]:
import serial
import time
import ctypes
import perlin_noise
import datetime
import random

import pyaudio
import math

In [81]:
# Set up serial communications
port = "/dev/cu.usbserial-1420"
baud = 115200
ser = serial.Serial(port, baud)
time.sleep(2)

In [65]:
# Create an instance of PyAudio
audio = pyaudio.PyAudio()

# Open a stream to capture audio from the microphone
stream = audio.open(format=pyaudio.paInt16,
                    channels=1,
                    rate=44100,
                    input=True,
                    frames_per_buffer=1024)

In [71]:
def serialize(motor, acceleration, speed, revolutions):
    motor_b = int(motor).to_bytes()
    acceleration_b = acceleration.to_bytes(2, signed = True)
    speed_b = speed.to_bytes(2, signed = True)
    revolutions_b = int(revolutions * 16.0 * 200.0).to_bytes(2, signed = True)
    print(( motor_b + acceleration_b + speed_b + revolutions_b).hex(' ', 2) )
    return motor_b + acceleration_b + speed_b + revolutions_b

def serialize_float(motor, acceleration, speed, revolutions):
    acceleration_32 = ctypes.c_float(acceleration)
    speed_32 = ctypes.c_float(speed)
    revolutions_32 = ctypes.c_float(revolutions)
    
    motor_b = int(motor).to_bytes()
    acceleration_b = bytes(acceleration_32)
    speed_b = bytes(speed_32)
    revolutions_b = bytes(revolutions_32)
    return (motor_b + acceleration_b + speed_b + revolutions_b)

def get_rms(frames):
    count = len(frames)/2.0
    sum_squares = 0.0
    sum_samples = 0.0

    for i in range(0, len(frames), 2):
        # Unpack the frame (2 bytes) into a signed integer
        n = int.from_bytes(frames[i:i+2], byteorder='little', signed=True)
        sum_squares += n ** 2
        sum_samples += n

    rms = math.sqrt(sum_squares / count)
    dc = sum_samples / count
    return rms
    return (rms, dc)

In [83]:
acceleration = 1000.0
moving_average = 0.0
moving_average_num = 10.0

while True:
    # Read audio data from the stream
    frames = b''
    for i in range(4):
        data = stream.read(1024, exception_on_overflow = False)
        frames += data

    # print("Finished recording.")
    rms_value = get_rms(frames)
    moving_average = (moving_average * moving_average_num + rms_value) / (moving_average_num + 1)
    # print(f"values: {rms_value}")

    myspeed = moving_average / 50.0
    print(myspeed)
    ser.write( serialize_float(motor = 0, acceleration = acceleration, speed = myspeed, revolutions = 1 ) )
    ser.write( serialize_float(motor = 1, acceleration = acceleration, speed = myspeed, revolutions = 1 ) )
    # ser.write( serialize_float(motor = 2, acceleration = acceleration, speed = myspeed, revolutions = 1 ) )
    # ser.write( serialize_float(motor = 3, acceleration = acceleration, speed = myspeed, revolutions = 1 ) )


0.16328112333914974
0.3484110825205475
0.4743794187988835
0.5873958210550917
0.699191300287036
0.7880920870452854
0.8770651790820652
0.9669468755384009
1.0331184432434488
1.1212483906214477
1.1864125966498178
1.2478862485503988
1.3170819287376963
1.4177318160362256
1.48391967092179
1.515540710351342
1.5580164693126772
1.5891257383802746
1.6015323235521957
1.629447123693072
1.6295753126726666
1.6383845934977885
1.636826188570064
1.6322353210279599
1.6315986677401286
1.6467641460452194
1.6660126287441748
1.673009448997874
1.6689615151355632
1.7038898442050554
1.839867786787228
2.0455022489969634
2.2214121694773397
2.3603945746557176
2.5448123586250784
2.7464129049230492
3.1252762815440693
3.6610378014980296
4.473413672557243
5.9646859833217025
7.044505853282448
7.95576433940215
8.80598888416321
9.385327299434815
9.726244345632521
9.981037478513935
10.536208709801876
10.933406640889496
11.166649252609652
11.612091030360798
11.999706221861663
12.48625963490543
12.826851137082386
12.9826223

KeyboardInterrupt: 

In [63]:
stream.stop_stream()
stream.close()
audio.terminate()